In [1]:
#!/home/ubuntu/.conda/envs/quixotic/bin/python

In [16]:
import pandas as pd
import os
import datetime
import pytz

import parameters

In [17]:
# TO DO: move DB parameters to file

import mysql.connector

DB_USER = parameters.DB_USER
DB_PASSWORD = parameters.DB_PASSWORD
DB_HOST = parameters.DB_HOST
DB_NAME = parameters.DB_NAME

In [18]:
DATA_DIR_RESCUE_TIME = parameters.DATA_DIR_RESCUE_TIME
DATA_DIR_EMAIL = parameters.DATA_DIR_EMAIL
DATA_DIR_CALENDAR = parameters.DATA_DIR_CALENDAR
ETL_DIR = parameters.ETL_DIR

USER_TIMEZONE = "US/Eastern"

In [23]:
df_rt = pd.DataFrame()
names = ['hour', 'duration', 'title', 'detail']

#file = 'zcarwile_2015-10-02.txt'
for file in os.listdir(DATA_DIR_RESCUE_TIME):
    df_tmp = pd.DataFrame()
    if ".txt" in file:
        full_path = '%s/%s' % (DATA_DIR_RESCUE_TIME, file)
        try:
            df_tmp = pd.read_table(full_path, sep='\t', header=None, names=names)
            df_rt = pd.concat([df_rt, df_tmp], ignore_index=True)
        except:
            print('Error processing: ' + full_path)
    
df_rt.tail()

,hour,duration,title,detail
21704,2017-01-06T18:00:00,248,play.spotify.com,No Details
21705,2017-01-07T12:00:00,1389,iterm2,No Details
21706,2017-01-07T12:00:00,241,python,Spyder (Python 2.7)
21707,2017-01-07T12:00:00,100,stackoverflow.com,load parameters from a file in Python - Stack ...
21708,2017-01-07T12:00:00,77,barrysbootcamp.com,No Details


In [24]:
# to make RT data "aware"
def to_aware(hour, timezone):
    est = pytz.timezone(timezone)
    aware = est.localize(datetime.datetime.strptime(hour,'%Y-%m-%dT%H:%M:%S'))
    return(aware)
                         
df_rt['start'] = df_rt.apply(lambda row: to_aware(row['hour'], USER_TIMEZONE), axis=1)
df_rt['start'] = pd.to_datetime(df_rt['start'], utc=True)

In [25]:
# create fake end time
# ultimately, rescue time is a poor data source because we're not using the raw tracks.  but ok for illustration
def create_end_time(hour, timezone, duration):
    est = pytz.timezone(timezone)
    tmp = est.localize(datetime.datetime.strptime(hour,'%Y-%m-%dT%H:%M:%S')) 
    duration = int(duration)
    end_time = tmp + datetime.timedelta(seconds=duration)
    return(end_time)

df_rt['end'] = df_rt.apply(lambda row: create_end_time(row['hour'], USER_TIMEZONE, row['duration']), axis=1)
df_rt['end'] = pd.to_datetime(df_rt['end'], utc=True)

df_rt.drop('hour', 1, inplace=True)
df_rt.drop('duration', 1, inplace=True)

In [26]:
df_rt.sort_values(by='start', inplace=True)
df_rt['tags'] = 'Rescue Time'

df_rt.tail()

,title,detail,start,end,tags
21704,play.spotify.com,No Details,2017-01-06 23:00:00,2017-01-06 23:04:08,Rescue Time
21707,stackoverflow.com,load parameters from a file in Python - Stack ...,2017-01-07 17:00:00,2017-01-07 17:01:40,Rescue Time
21705,iterm2,No Details,2017-01-07 17:00:00,2017-01-07 17:23:09,Rescue Time
21706,python,Spyder (Python 2.7),2017-01-07 17:00:00,2017-01-07 17:04:01,Rescue Time
21708,barrysbootcamp.com,No Details,2017-01-07 17:00:00,2017-01-07 17:01:17,Rescue Time


In [28]:
# Email
#file = 'zcarwile_1.txt'

df_email = pd.DataFrame()
names = ['detail','sent','features','title','redaction'] # TO DO: need an extra column -- for now I'm just stuffing

for file in os.listdir(DATA_DIR_EMAIL):
    if ".txt" in file:
        df_tmp = pd.DataFrame()
        full_path = '%s/%s' % (DATA_DIR_EMAIL, file)
        try:
            df_tmp = pd.read_table(full_path, header=None, names=names)
            df_email = pd.concat([df_email, df_tmp], ignore_index=True)
        except:
            print('Error processing: ' + full_path)
            
df_email['start'] = pd.to_datetime(df_email['sent'], utc=True)
df_email.drop('sent', 1, inplace=True)

df_email.sort_values(by='start', inplace=True)

df_email['tags'] = 'Gmail'
df_email.tail()

,detail,features,title,redaction,start,tags
101,15975e7189105579,ttolerico@continuum.io,"Accepted: Founder Friday @ Fri Jan 27, 2017 11...",Zachary Carwile has accepted this invitation. ...,2017-01-06 22:29:56,Gmail
100,15975e72dde2fe1b,Payal Cudia <pcudia@continuum.io>,Re: Your AnacondaCON Hotel Confirmation,Thanks! _______________________________ Zachar...,2017-01-06 22:30:01,Gmail
104,15976090ca414696,Zach Carwile <Zachary.carwile@gmail.com>,testing,_______________________________ Zachary Carwil...,2017-01-06 23:07:01,Gmail
103,1597666e7bd9d553,jamaral@continuum.io,Accepted: Continuum POC Call with Kaiser @ Tue...,Zachary Carwile has accepted this invitation. ...,2017-01-07 00:49:32,Gmail
102,15979c8a97009308,pcesarini@continuum.io,Accepted: Continuum Analytics /Govinvest Servi...,Zachary Carwile has accepted this invitation. ...,2017-01-07 16:35:10,Gmail


In [29]:
# Calendar
#file = 'zcarwile_1.txt'
df_calendar = pd.DataFrame()
names = ['detail','start_time','end_time','features','title']

for file in os.listdir(DATA_DIR_CALENDAR):
    if ".txt" in file:
        full_path = '%s/%s' % (DATA_DIR_CALENDAR, file)
        try:
            df_tmp = pd.read_table(full_path, header=None, names=names)
            df_calendar = pd.concat([df_calendar, df_tmp], ignore_index=True)
        except:
            print('Error processing: ' + full_path)


df_calendar['start'] = pd.to_datetime(df_calendar['start_time'], utc=True)
df_calendar['end'] = pd.to_datetime(df_calendar['end_time'], utc=True)

df_calendar.drop('start_time', 1, inplace=True)
df_calendar.drop('end_time', 1, inplace=True)

df_calendar.sort_values(by='start', inplace=True)
df_calendar['tags'] = 'Google Calendar'

df_calendar.tail()

,detail,features,title,start,end,tags
180,ngs1n79tsj3rimcv0l235nnkds,None,Small Improvements draft,2017-01-06 22:00:00,2017-01-06 23:00:00,Google Calendar
181,r18vlg88m6g23lu5a3o8i87mk4,None,relax,2017-01-06 23:30:00,2017-01-06 23:45:00,Google Calendar
182,23kcjej2iqe05ep3u37fnp5p9g,None,Dinner w/Rudy,2017-01-08 00:00:00,2017-01-08 03:00:00,Google Calendar
184,_60q30c1g60o30e1i60o4ac1g60rj8gpl88rj2c1h84s34...,Felipe Castillo Carvajal,AEN - Security touch point,2017-01-09 14:00:00,2017-01-09 15:00:00,Google Calendar
183,_60q30c1g60o30e1i60o4ac1g60rj8gpl88rj2c1h84s34...,None,Precall for Anaconda Installation,2017-01-09 14:00:00,2017-01-09 16:00:00,Google Calendar


In [30]:
# Check for unified data types in timestamp
#print(type(df_rt.loc[0]['start']))
#print(type(df_rt.loc[0]['start']))
#print(type(df_email.loc[0]['start']))
#print(type(df_calendar.loc[0]['start']))
#print(type(df_calendar.loc[0]['end']))

In [35]:

frames = [df_rt, df_email, df_calendar]

result = pd.concat(frames, ignore_index=True)
result.sort_values(by='start', inplace=True)
result['user_id'] = 1
result.tail(100)

,detail,end,features,redaction,start,tags,title,user_id
21649,Spyder (Python 3.4),2017-01-05 23:04:39,NaN,NaN,2017-01-05 23:00:00,Rescue Time,python,1
21642,No Details,2017-01-05 23:03:40,NaN,NaN,2017-01-05 23:00:00,Rescue Time,flowdock.com,1
21643,Users.messages | Gmail API | Google Develo...,2017-01-05 23:01:29,NaN,NaN,2017-01-05 23:00:00,Rescue Time,developers.google.com,1
21644,API Reference | Google Calendar API | Goog...,2017-01-05 23:01:45,NaN,NaN,2017-01-05 23:00:00,Rescue Time,developers.google.com,1
21645,Events | Google Calendar API | Google Deve...,2017-01-05 23:03:10,NaN,NaN,2017-01-05 23:00:00,Rescue Time,developers.google.com,1
21646,No Details,2017-01-05 23:03:52,NaN,NaN,2017-01-05 23:00:00,Rescue Time,Gmail,1
21647,Gmail API Overview | Gmail API | Google De...,2017-01-05 23:04:57,NaN,NaN,2017-01-05 23:00:00,Rescue Time,developers.google.com,1
21648,No Details,2017-01-05 23:07:05,NaN,NaN,2017-01-05 23:00:00,Rescue Time,ondemand.cloudera.com,1
21798,15970ed3c03ba251,NaT,"Kristopher Overholt <koverholt@continuum.io>, ...","Hi Kris, Christine: Charles Schwab wants our n...",2017-01-05 23:18:32,Gmail,REST API deployment,1
22806,h2tqa340jqocgpjjsrf7rl365g,2017-01-06 02:00:00,None,NaN,2017-01-05 23:30:00,Google Calendar,FI guys,1


In [36]:
# unified file with UTC timestamps

outfile = '%s/%s' % (ETL_DIR, 'zcarwile.txt')
result.to_csv(outfile, sep="\t")

In [37]:
# test for file pulled from EC2

#z = pd.read_csv('zcarwile.txt','\t')
#z.tail()

In [38]:
cnx = mysql.connector.connect(user=DB_USER, password=DB_PASSWORD, host=DB_HOST, db=DB_NAME)
result.to_sql('quixotic_api_event', cnx, flavor='mysql', if_exists='append', index=True, index_label='id')
cnx.close()

/Users/zcarwile/anaconda/envs/quixotic/lib/python3.4/site-packages/pandas/core/generic.py:1003: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  dtype=dtype)


In [ ]:
# HOW DO I initialize table in Django with a default user
#insert into quixotic_api_user (name) values ('zcarwile');